In [1]:
import pandas as pd
import numpy as np
import glob

A base de dívida ativa servirá para rotular as empresas que estão inscritas na dívida e também as empresas que possuem sócios inscritos em dívida. 
Para esse propósito de rotulagem, precisamos apenas das colunas CPF_CNPJ e NOME_DEVEDOR. Esta é necessária porque o CPF informado não é completo e será necessário o nome da pessoa física para junção com outras bases. Não é necessário distinguir o tipo de dívida.

In [2]:
fields = ["CPF_CNPJ", "NOME_DEVEDOR"]
fields_dtype = {"CPF_CNPJ": "str", "NOME_DEVEDOR": "str"}

In [3]:
# IMPORTAÇÃO DOS ARQUIVOS CSV DA DIVIDA NÃO PREVIDENCIÁRIA

path = r'C:\DATASETS\Divida_Ativa\Nao_Previdenciario'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    temp_df = pd.read_csv(filename, index_col=None, header=0, sep=';', encoding='ANSI', dtype=fields_dtype, usecols=fields)
    li.append(temp_df)

# Concatena o resultado no dataframe
divida_df = pd.concat(li, axis=0, ignore_index=True)

divida_df.describe

<bound method NDFrame.describe of                     CPF_CNPJ                                NOME_DEVEDOR
0               XXX464.712XX            ERISVANDO TORQUATO DO NASCIMENTO
1         04.718.079/0001-41                                 I. B. SOUSA
2               XXX468.172XX                    CLAUDIO BRANDAO DA SILVA
3         05.395.942/0001-30          DISTRIBUIDORA VALE DO GUAPORE LTDA
4               XXX117.482XX             SEBASTIAO PEREIRA DO NASCIMENTO
...                      ...                                         ...
18001159  02.877.801/0001-65        AGROPALMAS AGROPECUARIA DO PALMAS SA
18001160        XXX955.691XX                MARIA COTINHA VIEIRA RIBEIRO
18001161        XXX372.269XX                               ALCIDES SERPA
18001162  07.272.618/0001-50  LDM Consultoria em Gestao Empresarial LTDA
18001163  36.999.985/0001-58     PETROLIMA COMERCIO DE COMBUSTIVEIS LTDA

[18001164 rows x 2 columns]>

In [4]:
# IMPORTAÇÃO DOS ARQUIVOS CSV DA DIVIDA FGTS

path = r'C:\DATASETS\Divida_Ativa\FGTS'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    temp_df = pd.read_csv(filename, index_col=None, header=0, sep=';', encoding='ANSI', dtype=fields_dtype, usecols=fields)
    li.append(temp_df)

# Concatena o resultado no dataframe
divida_df = pd.concat([divida_df, pd.concat(li)], ignore_index=True)
                       
divida_df.describe

<bound method NDFrame.describe of                     CPF_CNPJ                               NOME_DEVEDOR
0               XXX464.712XX           ERISVANDO TORQUATO DO NASCIMENTO
1         04.718.079/0001-41                                I. B. SOUSA
2               XXX468.172XX                   CLAUDIO BRANDAO DA SILVA
3         05.395.942/0001-30         DISTRIBUIDORA VALE DO GUAPORE LTDA
4               XXX117.482XX            SEBASTIAO PEREIRA DO NASCIMENTO
...                      ...                                        ...
18436109  00.770.420/0001-57        BALTAZAR ANTONIO DA SILVA O MINEIRO
18436110  04.769.913/0001-28                       MOURA & REZENDE LTDA
18436111  03.389.184/0001-11                          S. G. JORGE COSTA
18436112  03.389.184/0001-11                          S. G. JORGE COSTA
18436113  26.632.547/0001-01  ADRIANO F DA SILVA PRODUTOS FARMACEUTICOS

[18436114 rows x 2 columns]>

In [5]:
# IMPORTAÇÃO DOS ARQUIVOS CSV DA DIVIDA PREVIDENCIÁRIA

path = r'C:\DATASETS\Divida_Ativa\Previdenciario'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    temp_df = pd.read_csv(filename, index_col=None, header=0, sep=';', encoding='ANSI', dtype=fields_dtype, usecols=fields)
    li.append(temp_df)

# Concatena o resultado no dataframe
divida_df = pd.concat([divida_df, pd.concat(li)], ignore_index=True, sort=False)
                       
divida_df.describe

<bound method NDFrame.describe of                     CPF_CNPJ  \
0               XXX464.712XX   
1         04.718.079/0001-41   
2               XXX468.172XX   
3         05.395.942/0001-30   
4               XXX117.482XX   
...                      ...   
23508566  13.758.946/0001-63   
23508567  12.967.577/0001-56   
23508568  25.086.695/0001-05   
23508569  25.086.695/0001-05   
23508570  25.086.695/0001-05   

                                               NOME_DEVEDOR  
0                          ERISVANDO TORQUATO DO NASCIMENTO  
1                                               I. B. SOUSA  
2                                  CLAUDIO BRANDAO DA SILVA  
3                        DISTRIBUIDORA VALE DO GUAPORE LTDA  
4                           SEBASTIAO PEREIRA DO NASCIMENTO  
...                                                     ...  
23508566                          ROSANA MESQUITA FERNANDES  
23508567  AM E FILHOS COMERCIO DE MATERIAIS PARA CONSTRU...  
23508568               

Observou-se que não foram gerados DtypeWarnings na importação dos arquivos, confirmando o mapeamento correto para os tipos das colunas, conforme analisado via dicionário de dados da base.

## Iniciando o processamento e tratamento dos dados:

#### Obtendo mais informações sobre os dados:

In [6]:
divida_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23508571 entries, 0 to 23508570
Data columns (total 2 columns):
CPF_CNPJ        object
NOME_DEVEDOR    object
dtypes: object(2)
memory usage: 358.7+ MB


#### Identificando percentual de valores ausentes por coluna:

In [7]:
(divida_df.isnull().sum() / divida_df.shape[0] * 100).sort_values(ascending=False)

NOME_DEVEDOR    0.0
CPF_CNPJ        0.0
dtype: float64

Percebe-se que a base é bem completa, sem valores ausentes nas colunas selecionadas.

#### Verificando quantidade de valores únicos em cada coluna:

In [8]:
divida_df.nunique().sort_values()

CPF_CNPJ        3998123
NOME_DEVEDOR    5141363
dtype: int64

Como o objetivo dessa base processada é apenas para indicar se uma pessoa, física ou jurídica, consta como devedor da união, então podemos remover as tuplas duplicadas:

In [11]:
divida_df.drop_duplicates(inplace=True)

In [17]:
divida_df.describe

<bound method NDFrame.describe of             index            CPF_CNPJ  \
0               0        XXX464.712XX   
1               1  04.718.079/0001-41   
2               2        XXX468.172XX   
3               3  05.395.942/0001-30   
4               4        XXX117.482XX   
...           ...                 ...   
5994589  23508539  11.966.821/0001-01   
5994590  23508561  38.135.935/0001-58   
5994591  23508566  13.758.946/0001-63   
5994592  23508567  12.967.577/0001-56   
5994593  23508568  25.086.695/0001-05   

                                              NOME_DEVEDOR  
0                         ERISVANDO TORQUATO DO NASCIMENTO  
1                                              I. B. SOUSA  
2                                 CLAUDIO BRANDAO DA SILVA  
3                       DISTRIBUIDORA VALE DO GUAPORE LTDA  
4                          SEBASTIAO PEREIRA DO NASCIMENTO  
...                                                    ...  
5994589                       WANDERVAL APAREC

In [16]:
divida_df.reset_index(inplace=True)

In [19]:
divida_df.drop('index', axis=1, inplace=True)

In [20]:
divida_df.describe

<bound method NDFrame.describe of                    CPF_CNPJ                                       NOME_DEVEDOR
0              XXX464.712XX                   ERISVANDO TORQUATO DO NASCIMENTO
1        04.718.079/0001-41                                        I. B. SOUSA
2              XXX468.172XX                           CLAUDIO BRANDAO DA SILVA
3        05.395.942/0001-30                 DISTRIBUIDORA VALE DO GUAPORE LTDA
4              XXX117.482XX                    SEBASTIAO PEREIRA DO NASCIMENTO
...                     ...                                                ...
5994589  11.966.821/0001-01                       WANDERVAL APARECIDO DE FARIA
5994590  38.135.935/0001-58                  MIL TOPOGRAFIA E CONSULTORIA LTDA
5994591  13.758.946/0001-63                          ROSANA MESQUITA FERNANDES
5994592  12.967.577/0001-56  AM E FILHOS COMERCIO DE MATERIAIS PARA CONSTRU...
5994593  25.086.695/0001-05                            JOSE BONIFACIO SOBRINHO

[5994594 rows x 2

##### Limpeza dos dados

Na coluna CPF_CNPJ: 1) remover pontuação e barras; 2) para os casos de CPF, trocar X por * para ficar compatível com a base de sócios e facilitar as junções.

In [22]:
divida_df["CPF_CNPJ"] = divida_df["CPF_CNPJ"].str.replace('[\.\-\/]', '', regex=True).str.replace('X', '*', regex=False)

In [23]:
divida_df.describe

<bound method NDFrame.describe of                CPF_CNPJ                                       NOME_DEVEDOR
0           ***464712**                   ERISVANDO TORQUATO DO NASCIMENTO
1        04718079000141                                        I. B. SOUSA
2           ***468172**                           CLAUDIO BRANDAO DA SILVA
3        05395942000130                 DISTRIBUIDORA VALE DO GUAPORE LTDA
4           ***117482**                    SEBASTIAO PEREIRA DO NASCIMENTO
...                 ...                                                ...
5994589  11966821000101                       WANDERVAL APARECIDO DE FARIA
5994590  38135935000158                  MIL TOPOGRAFIA E CONSULTORIA LTDA
5994591  13758946000163                          ROSANA MESQUITA FERNANDES
5994592  12967577000156  AM E FILHOS COMERCIO DE MATERIAIS PARA CONSTRU...
5994593  25086695000105                            JOSE BONIFACIO SOBRINHO

[5994594 rows x 2 columns]>

In [24]:
divida_df.nunique().sort_values()

CPF_CNPJ        3998123
NOME_DEVEDOR    5141363
dtype: int64

Percebe-se que mantivemos as mesmas quantidades de valores únicos.

Salvando resultado:

In [25]:
divida_df.to_csv(r'C:\DATASETS\Divida_Ativa\divida_ativa_todos.csv', index = False)